In [16]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn import tree
from sklearn.cluster import DBSCAN
import sklearn.mixture
import sklearn.datasets

tbl = pd.read_csv('neowise.csv')
truth = pd.read_csv('allwise.csv')

# tbl = pd.read_csv('neowise1.csv')
# truth = pd.read_csv('allwise1.csv')

# tbl = pd.read_csv('transient_test.csv')
# truth = pd.read_csv('transient_test_truth.csv')



truth["mag"] = 22.5 - 2.5 * np.log10(truth["flux_1"])
print(truth.sort_values(by='mag', ascending=True).head(10))
truth = truth[truth['mag'] < 15.5]

xmatch_radius = 4/3600 # deg

def plot(tbl, cluster=False):
    truth_pos = truth[['ra', 'dec']].to_numpy()
    fig = go.Figure(go.Scatter(x=tbl['ra'], y=tbl['dec'], mode='markers', marker=dict(size=3, opacity=0.5, color=("blue" if not cluster else tbl["cluster"]))))
    # fig.add_trace(go.Scatter(x=truth_pos[:, 0], y=truth_pos[:, 1], mode='markers', marker=dict(size=5 * (16 / truth["mag"]), opacity=0.7, color='red')))

    for x,y in truth_pos:
        fig.add_shape(type="circle", x0=x-xmatch_radius, y0=y-xmatch_radius, x1=x+xmatch_radius, y1=y+xmatch_radius, line=dict(color="red"))



    fig.update_layout(height=900, width=900, title_text='SKY')
    fig.show()

def xmatch_count(tbl, truth):=
    count = 0
    x = np.array(tbl["ra"])
    y = np.array(tbl["dec"])
    t = truth[["ra", "dec"]].to_numpy()

    inclusions = np.array([False] * len(x))

    log = []

    for xt, yt in t:
        d = np.power(x - xt, 2) + np.power(y - yt, 2)
        incl = d < xmatch_radius**2

        inclusions = inclusions | incl

        log.append(np.sum(incl))

    count = np.sum(inclusions)

    return count

print("NOISE LEVEL:")
w1f = 309.54 * 10**(-tbl["w1mpro"] / 2.5)

n = (w1f * tbl["w1snr"]).mean()

print("Proposed eps")
eps = 2**(-1-np.log10(n/6))
print(eps)

     Unnamed: 0          ra        dec        flux_1        mag
164         164  246.351114 -23.460324  5.898989e+06   5.573056
69           69  246.351114 -23.460316  5.824790e+06   5.586799
40           40  246.340965 -23.501879  1.563581e+05   9.514699
35           35  246.348913 -23.510812  1.301303e+05   9.714054
159         159  246.369931 -23.478553  7.544352e+04  10.305945
62           62  246.369934 -23.478553  7.507615e+04  10.311245
163         163  246.336991 -23.469887  6.854405e+04  10.410076
67           67  246.336994 -23.469884  6.841781e+04  10.412077
162         162  246.348497 -23.470083  4.891680e+04  10.776355
66           66  246.348506 -23.470100  4.855020e+04  10.784522
NOISE LEVEL:
Proposed eps
1.0137934882069781


In [2]:
# FILTERS #
filtered_tbl = tbl.copy()
filtered_tbl = filtered_tbl[np.isnan(filtered_tbl["w1sigmpro"]) == False]
print("Initial pts: ", len(filtered_tbl))
xm_1 = xmatch_count(filtered_tbl, truth)
l1 = len(filtered_tbl)
print("Initial xmatch: ", xm_1)
plot(filtered_tbl)


# South Atlantic Anomaly and Chi-Squared - Deals with cosmic rays
# chi_quantile_regular = tbl["w1rchi2"].quantile(0.95) # Expected proportion of non-cosmic ray sources outside of the SAA
# chi_quantile_saa = tbl["w1rchi2"].quantile(0.8) # Expected proportion of non-cosmic ray sources inside the SAA
# # print(f"Chi-Squared Quantile (Regular): {chi_quantile_regular}")
# # print(f"Chi-Squared Quantile (SAA): {chi_quantile_saa}")

# chi_quantile = [chi_quantile_saa if saasep < 5.0 else chi_quantile_regular for saasep in filtered_tbl["saa_sep"]]
# filtered_tbl = filtered_tbl[filtered_tbl["w1rchi2"] < 8]
# print(f"Chi2 Filter: {l1} -> {len(filtered_tbl)} = {(len(filtered_tbl) - l1) / l1 * 100:.2f}%")
# print(f"{(xmatch_count(filtered_tbl, truth) - xm_1) / xm_1 * 100:.2f}% lost (xmatch)")
# plot(filtered_tbl)

# Real Detection

# Artifacts
criterion = np.array([flags[0].lower() != 'd' and flags[0] != 'P' and flags[0] != 'O' for flags in filtered_tbl["cc_flags"]])
filtered_tbl = filtered_tbl[criterion]
# filtered_tbl = filtered_tbl[filtered_tbl["cc_flags"] == "0000"]
print(f"Artifact Filter: {l1} -> {len(filtered_tbl)} = {(len(filtered_tbl) - l1) / l1 * 100:.2f}%")
print(f"{(xmatch_count(filtered_tbl, truth) - xm_1) / xm_1 * 100:.2f}% lost (xmatch)")
plot(filtered_tbl)

# SNR
filtered_tbl = filtered_tbl[filtered_tbl["w1snr"] > 4]
print(f"SNR Filter: {l1} -> {len(filtered_tbl)} = {(len(filtered_tbl) - l1) / l1 * 100:.2f}%")
print(f"{(xmatch_count(filtered_tbl, truth) - xm_1) / xm_1 * 100:.2f}% lost (xmatch)")
plot(filtered_tbl)

print("Final length: ", len(filtered_tbl))
print("Final xmatch: ", xmatch_count(filtered_tbl, truth))

Initial pts:  17365
Initial xmatch:  6059


Artifact Filter: 17365 -> 16266 = -6.33%
-3.14% lost (xmatch)


SNR Filter: 17365 -> 10877 = -37.36%
-8.07% lost (xmatch)


Final length:  10877
Final xmatch:  5570


In [13]:
dbscan = DBSCAN(eps=1/3600, min_samples=10)
dbscan.fit(filtered_tbl[["ra", "dec"]])
labels = dbscan.labels_
clustered = filtered_tbl[labels != -1]
clustered["cluster"] = labels[labels != -1]
clusters = {}
for i, c in enumerate(labels):
    if c not in clusters:
        clusters[c] = []
    clusters[c].append(i)

plot(clustered, cluster=True)
cluster_num = 24
plot(clustered[clustered["cluster"]==cluster_num])


/var/folders/vx/04hztcd11rj6ly3sf9hwtt080000gn/T/ipykernel_33354/4131387181.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[[1.         0.13545252]
 [0.13545252 0.96839688]]


In [ ]:
def loss(eps, minpts):
    x1 = xmatch_count(filtered_tbl, truth)
    dbscan = DBSCAN(eps=eps, min_samples=minpts)
    dbscan.fit(filtered_tbl[["ra", "dec"]])
    l = dbscan.labels_
    t = filtered_tbl[l != -1]
    x2 = xmatch_count(t, truth)

    not_in = len(t) - x2

    return 0.3*((x1 - x2) / x1) + (not_in / len(t))

# gridsearch for optimal parameters for DBSCAN
eps = np.linspace(1, 50, 80)
minpts = np.linspace(1, 30, 30)
losses = np.zeros((len(eps), len(minpts)))

for i, e in enumerate(eps):
    for j, m in enumerate(minpts):
        losses[i, j] = loss(e/3600, int(m))

plt.imshow(losses, extent=(minpts[0], minpts[-1], eps[0], eps[-1]), aspect='auto')
plt.colorbar()


